## 수익률 계산

In [1]:
from pykrx import stock

# from pandas_datareader import data as pdr
import yfinance as yf

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

#### 벤치마크 수익률

In [2]:
# 코스피
kospi = stock.get_index_fundamental('20130401',"20230331", '1001')[['종가']]
kospi['수익률']=kospi['종가'].pct_change().fillna(0)

kospi

,종가,수익률
날짜,,
2013-04-01,1995.99,0.000000
2013-04-02,1986.15,-0.004930
2013-04-03,1983.22,-0.001475
2013-04-04,1959.45,-0.011986
2013-04-05,1927.23,-0.016443
...,...,...
2023-03-27,2409.22,-0.002377
2023-03-28,2434.94,0.010676
2023-03-29,2443.92,0.003688


In [3]:
# 코스닥
kosdaq=stock.get_index_fundamental('20130401',"20230331", '2001')[['종가']]
kosdaq['수익률']=kosdaq['종가'].pct_change().fillna(0)

kosdaq

,종가,수익률
날짜,,
2013-04-01,553.97,0.000000
2013-04-02,552.83,-0.002058
2013-04-03,557.96,0.009280
2013-04-04,555.23,-0.004893
2013-04-05,547.51,-0.013904
...,...,...
2023-03-27,827.69,0.004344
2023-03-28,833.51,0.007032
2023-03-29,843.94,0.012513


In [4]:
# S&P500
# 해당코드는 31일까지 해버리면 31일은 안나옴. 즉, 4월 1일까지라고 해야 3월 31일까지 나옴
# 시작도 동일함 31일부터 해야 4월 1일꺼부터 시작임. 31일은 미포함
sp = yf.download('^GSPC', '2013-03-31','2023-04-01')[['Adj Close']]
sp['수익률'] = sp['Adj Close'].pct_change().fillna(0)

sp

[*********************100%%**********************]  1 of 1 completed


,Adj Close,수익률
Date,,
2013-04-01,1562.170044,0.000000
2013-04-02,1570.250000,0.005172
2013-04-03,1553.689941,-0.010546
2013-04-04,1559.979980,0.004048
2013-04-05,1553.280029,-0.004295
...,...,...
2023-03-27,3977.530029,0.001647
2023-03-28,3971.270020,-0.001574
2023-03-29,4027.810059,0.014237


#### 결합

In [6]:
rtns = pd.DataFrame()

benchmarks = [kospi, kosdaq, sp]
bname = ['kospi', 'kosdaq', 'S&P500']

for rtn, name in zip(benchmarks, bname):
    rtn = rtn.rename(columns={'수익률' : name})
    rtns = pd.merge(
        rtns, rtn[[name]],
        left_index=True, right_index=True,
        how='outer'
    )

rtns = rtns.fillna(0)

rtns.head()

,kospi,kosdaq,S&P500
2013-04-01,0.000000,0.000000,0.000000
2013-04-02,-0.004930,-0.002058,0.005172
2013-04-03,-0.001475,0.009280,-0.010546
2013-04-04,-0.011986,-0.004893,0.004048
2013-04-05,-0.016443,-0.013904,-0.004295


In [7]:
rtns.shape

(2595, 3)

In [8]:
rtns.isna().sum()

kospi     0
kosdaq    0
S&P500    0
dtype: int64

In [9]:
rtns.index

DatetimeIndex(['2013-04-01', '2013-04-02', '2013-04-03', '2013-04-04',
               '2013-04-05', '2013-04-08', '2013-04-09', '2013-04-10',
               '2013-04-11', '2013-04-12',
               ...
               '2023-03-20', '2023-03-21', '2023-03-22', '2023-03-23',
               '2023-03-24', '2023-03-27', '2023-03-28', '2023-03-29',
               '2023-03-30', '2023-03-31'],
              dtype='datetime64[ns]', length=2595, freq=None)

### 파일로 저장

In [10]:
rtns.to_csv('benchmark/rtns_benchmark.csv')